In [ ]:
# Preparing

In [2]:
%matplotlib inline
from numpy import *
from matplotlib.pyplot import *

n = 33
h = 1./(n-1)

x=linspace(0,1,n)

a = -ones((n-1,)) # Offdiagonal entries
b = 2*ones((n,)) # Diagonal entries
A = (diag(a, -1) + diag(b, 0) + diag(a, +1))
A /= h**2
f = x*(1.-x)

# Change first row of the matrix A
A[0,:] = 0
A[:,0] = 0
A[0,0] = 1
f[0] = 0

# Change last row of the matrix A
A[-1,:] = 0
A[:,-1] = 0
A[-1,-1] = 1
f[-1] = 0

# Solution by direct method
u = linalg.solve(A, f)

# Jacobi 

$$ 
x_i^{k+1} = \frac{1}{A_{ii}} \times \left(b_i - \sum_{j\neq i} a_{ij}x_j^k\right)
$$


In [7]:
def jacobi(A, b, nmax=10000, eps=1e-10):
    cnt = 0
    n_unknown = len(A) # Number of incognites
    # Extract the diagonal and residual
    D_inv = zeros((n_unknown, n_unknown))
    R = array(A)
    tol = eps + 1
    for i in range(n_unknown):
        D_inv[i][i] = 1.0/A[i][i]
        R[i][i] = 0
    x_i = zeros_like(b)
    while cnt < nmax and tol > eps:
        x_i_n = D_inv.dot(b - R.dot(x_i))
        # Stopping by accomplishing the epsilon in all vars
        res = b - A.dot(x_i_n)
        tol = linalg.norm(res,2)
        x_i = x_i_n.copy()
        cnt += 1
    print("Done with: ", cnt)
    return x_i

sol_jacobi = jacobi(A, f)
print(linalg.norm(sol_jacobi - u)/linalg.norm(u))

Done with:  4777
9.682097967515382e-11


## Gauss-Seidel

$$ 
x_i^{k+1} = \frac{1}{A_{ii}} \times \left(b_i - \sum_{j=0}^{i-1} a_{ij}x_j^{k+1} - \sum_{j=i+1}^{N} a_{ij}x_j^k\right)
$$

In [29]:
def gauss_seidel(A,b,nmax=10000, eps=1e-10):
    cnt = 0
    n_unknown = len(A) # Number of incognites
    # Extract the diagonal and residual
    D = diag(A)
    D_inv = 1/D
    tol = eps + 1
    # Buffer
    x = zeros_like(b)
    x_new = zeros_like(b)
    while cnt < nmax and tol > eps:
        for i in range(n_unknown):
            x_new[i] = D_inv[i] * (b[i] - dot(A[i][:i],x_new[:i]) - dot(A[i][i+1:n_unknown],x[i+1:n_unknown]))
        # Stop condition
        res = b - dot(A,x_new)
        tol = linalg.norm(res,2)
        x = x_new.copy()
        cnt += 1
    print("Done with: ", cnt)
    return x

sol_gauss_seidel = gauss_seidel(A, f)
print(linalg.norm(sol_gauss_seidel - u)/linalg.norm(u))

Done with:  2390
9.577313808022264e-11


   ## Gradient method
   $$
   {\bf r}^k = {\bf b} - A {\bf x}^k
   $$
   
   $$
   \alpha^k = \frac{{\bf r}^{k^{T}} {\bf r}^k}{{\bf r}^{k^{T}} A{\bf r}^k}
   $$
   
   $$
   {\bf x}^{k+1} = {\bf x}^k + \alpha^k {\bf r}^k
   $$
   
   ### Preconditioned gradient method
   $$
   P{\bf z}^k =  {\bf r}^k
   $$
   
   $$
   \alpha^k = \frac{{\bf z}^{k^{T}} {\bf r}^k}{{\bf z}^{k^{T}} A{\bf z}^k}
   $$
 
   $$
   {\bf x}^{k+1} = {\bf x}^k + \alpha^k {\bf z}^k
   $$ 
   
   $$
   {\bf r}^{k+1} = {\bf r}^k  - \alpha^k A{\bf z}^k
   $$

In [34]:
def gradient(A, b, P, nmax=8000, eps=1e-10):
    X = zeros_like(b)
    X_n = zeros_like(b)
    R = b - A.dot(X)
    cnt = 0
    tol = eps + 1
    while cnt < nmax and tol > eps:
        Z = linalg.solve(P,R)
        alpha = (Z.T.dot(R)) / (Z.T.dot(A).dot(Z))
        X += alpha * Z
        R -= alpha * A.dot(Z)
        # Stop
        tol = linalg.norm(R,2)
        cnt += 1
    print("Done with: ", cnt)
    return X

sol_gradient = gradient(A, f, identity(len(A)))
print(linalg.norm(sol_gradient - u)/linalg.norm(u))
sol_preconditioned_gradient = gradient(A, f, A)
print(linalg.norm(sol_preconditioned_gradient - u)/linalg.norm(u))

Done with:  3909
7.095304299230852e-11
Done with:  1
2.6880576072308357e-15
